In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
data = pd.read_csv('/content/emoji_data.csv',header = None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [ ]:
pip install emoji

In [ ]:
import emoji

In [ ]:
emoji.emojize(':red_heart:')

'❤️'

In [ ]:
emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}


def label_emoji(label):
  return emoji.emojize(emoji_dict[label])

In [ ]:
x = data[0].values
y = data[1].values

#Embedding

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2024-12-16 20:32:01--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-12-16 20:32:01--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-12-16 20:32:02--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

In [ ]:
from zipfile import ZipFile
with ZipFile('/content/glove.6B.zip') as zip:
  zip.extractall()
  zip.close()

KeyboardInterrupt: 

In [ ]:
file = open('/content/glove.6B.100d.txt' , 'r' , encoding = 'utf8')
content = file.readlines()
file.close()
content


In [ ]:
content[0].split()[0]

'the'

In [ ]:
# this we got word embedding for differnt words
# now create a dict which will map them

embeddings = {}
for line in content:
  line = line.split()
  embeddings[line[0]] = np.array(line[1:])

In [ ]:
# we cant directly pass the input as a word so we will convert them into token
# also convert y into one-hot vector
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
word2index = tokenizer.word_index

In [ ]:
x_tokens = tokenizer.texts_to_sequences(x)
x_tokens[0:5]

[[103, 104, 3, 6, 105], [106, 3, 107], [1, 7, 108], [109, 4, 35], [36, 30]]

In [ ]:
# in NN we need to pass a uniform size input so we'll use padding to make them of same size
# each will be equal to max length in the dataset
def max_length(x):
  maxlen = 0
  for sent in x:
    maxlen = max(maxlen , len(sent))
  return maxlen

maxlen = max_length(x_tokens)
maxlen

10

In [ ]:
# so we'll pad all the sentences upto lenght 10
x_train = pad_sequences(x_tokens , maxlen = maxlen , padding = 'post' , truncating='post')
# padding--> post mean after last word , truncating--> mean if len of sent = 12 so shorten it from last

In [ ]:
x_train[0]

array([103, 104,   3,   6, 105,   0,   0,   0,   0,   0], dtype=int32)

In [ ]:
y = ['0' if label == '0v2' else label for label in y]

In [ ]:
y_train = to_categorical(y)
y_train[0]

array([0., 0., 0., 0., 1.])

#Model

In [ ]:
# weights wants the input in form of a matrix where number of rows = size of vocab , col = size of word embedding vector

embedding_matrix = np.zeros((len(word2index)+1 , embed_size))

for word , i in word2index.items():
  embed_vector = embeddings[word]
  embedding_matrix[i] = embed_vector

In [ ]:
embed_size = 100 #size of model usign here 100
model = Sequential()
model.add(Embedding(input_dim = len(word2index)+1 , output_dim = embed_size , input_length = maxlen, weights=embedding_matrix , trainable = False))
model.add(LSTM(units = 16 , return_sequences = True))
model.add(LSTM(units = 4))
model.add(Dense(5 , activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.fit(x_train, y_train, epochs = 100)


Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.2589 - loss: 1.5913
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3450 - loss: 1.5439
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3609 - loss: 1.5292
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3674 - loss: 1.5130 
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3541 - loss: 1.4898 
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3715 - loss: 1.4855
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4055 - loss: 1.4749
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4331 - loss: 1.4501 
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4948 - loss: 1.4203
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5120 - loss: 1.3829
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5311 - loss: 1.3567 
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5258 - los

In [ ]:
test = ["I feel good", "I feel very bad", "lets eat dinner"]

test_seq = tokenizer.texts_to_sequences(test)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis = 1)

for i in range(len(test)):
    print(test[i], label_emoji(y_pred[i]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
I feel good 😃
I feel very bad 😞
lets eat dinner 🍽️
